In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Project 1: Data analysis

## Bayesian approch

In order to predict the effect of genes on symptoms, we will use a simple Bayesian model. We consider these hypothesis:

$$\mu_0: P(symptoms|genes) = P(symptoms)$$
$$\mu_1: P(symptoms|genes) \neq P(symptoms)$$

In order words, does the genes have any effect on the symptoms or not? If it does, which genomes are the most relevant for the preventing the symptoms? 

We let Y be the symptoms and our X the genes. Y is an 8 bit vector for each observation, with 1 or zero for each bit. X is an 128 with the same charectaristic as Y.

The probability distribution of $ Y $ depends on an unknown parameter that we assume to be stochastic. We call this unknown parameter $\theta$, or in our program, $ p $. 

The formulation under is an alternative formulation of the the previous hypotesis. Does the parameter that the probability distribution of Y depend on, depend on X or not. $ \mu_0 $, our null, says that it does not. $\mu_1 $, the alternative hypothesis claims it does. We assume the Beta-Bernoulli model:
 
$$\mu_0: \theta_0 \sim Beta(\alpha_0 , \beta_0) ,  \ \ \  Y | \theta_0  $$
$$\mu_1: \theta_{1,x_t} \sim Beta(\alpha_{1,x_t} , \beta_{1,x_t}) , \ \ \ y_t | x_t \sim bernoulli(\theta_{1,x_t}) $$
 

Given the data, D, we try to estimate

$$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_1}(D) \cdot \xi(\mu_1)} $$

For simplicity, let $\mu = \mu_0$ or $\mu = \mu_1$. We define the posterior belief:

Let $P_{\mu} (D) = P(\mu |D)$ for $D = (x_t, y_t,x_{t-1},y_{t-1})_{t=1} ^T $ for a spesific gene X, and let $x_t$ be the value (0  or 1) of the gene for one observation, t, then $x_{t-1}$ is a vector of all the values for the previous observations including t-1. Further, let $y_t$ be the the value (0 or 1) of symptom Y for observation t defined in the same way. We assume that the different observations are independent. That is, 

$$ \begin{align*}P_{\mu}(D) & = P(y_1,...,y_t \cup x_1,...,x_t) \\ 
              & = \prod_{i=1}^{t} P(y_i | x_i,x_{i-1} , y_{i-1}) \end{align*} $$

With these assumptions, we can estimate $P_{\mu} (D)$: 
$$ P(y_t | x_t,x_{t-1} , y_{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x_{t-1},y_{t-1})$$

This makes adding new observations and updating beliefs easy. It works if we assume that a new observation is independent of the previous.

This estimate is called the marginal likelihood (posterior predictive distribution). It is a compound distribution. The probabilty $P(y_t | x_{t})$ is stochastic in $\theta$, so we have to integrate over all values of $\theta$. Every time a new observation is added we compute $$P(y_t | x_{t},x_{t-1}, y_{t-1})$$, and recalculate $$ \prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1}) $$. 

When calculating $$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $$ we are sequentially updating our belief every time a new observation is added. For every update $\xi(\mu)$, $P(\mu | D)$ is from the previous iteration, exept for in the very first iteration when $\xi(\mu)$ will cancel because it's constant.

This integral simplifies to the expectation of the posterior:

$$ P(y_t | x_{t},x_{t-1},y_{t-1}) = \int_{\theta} P_{\theta}(y_t) \cdot d \xi(\theta|y_{t-1}) = \frac{\alpha_{t,x_t}}{\alpha_{t,x_t} + \beta_{t,x_t}}.$$

Hence, we will not be computing the integral. Instead, we will update $\alpha$ and $\beta$ in a way that depends on the data. 

Now, focusing on $\mu_1$, we will use the Beta-Bernoulli conjugate prior. It is a Bernoulli distribution, a binomial with $n=1$, where the probability of success at each trial is a random variable. This random variable is assumed to be beta distributed. Since Y is either 1 or 0 it is reasonable to assume that $Y|X$ is Bernoulli distributed.

It is also convinient because of its conjugate prior property. We can now calculate $P( D | \mu_1) = \prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1}) $ beacuse we know that the marginal likelihood (posterior predictive distribution) is composed of a Bernoulli distribution and a beta distribution $ P(y_t | x_t , y_{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x_{t-1},y_{t-1})$. For each iteration we get that $P_{\mu} (D)$ is itself beta distributed.

The same integral calculation is done for $P(D | \mu_0)$ but now the marginal likelihood is assumed to be independent of the data.

For convenience, instead of calculating $P(D |\mu_1)$ directly, we will calculate $log(P(D|\mu_1))$, and in the end do $e^{log(P(D|\mu_1))}$ to obtain the desired probability.

The calculation is done sequentially. For each iteration we have a new observation, a new set of genes. We continously update the posterior belief, and put this into a list. The belief gets updated through calculating the marginal likelihood in each iteration, and then taking the product between this marginal likelihood and all the previous marginal likelihoods. In the end we obtain the estimate $$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $$.

For each gene we then reject the null if $P(\mu_1|D) > s$ for a given threshold $0 < s < 1$.


## The effect
Once we have the most relevant genes we will look at the individual effect of each gene on symptoms, i.e the probability of having a symptom given that you have one of these genes. 

This will be done in a somewhat simplistic fashion. As described above, we assume a Beta-Bernoulli conjugate prior that leads to a beta distributed posterior belief with parameters $\alpha$ and $\beta$. We will update $\alpha$ and $\beta$ sequentially when reading through the data. For a given gene $x_i$ and symptom $y_i$, if $x_i = 1$, $\alpha_i = \alpha_{i-1} + 1$ if $y_i = 1$ and $\beta_i = \beta_{i-1} + 1$ if $y_i = 0$. This way we get the expected value of our posterior $$\hat{p} = \frac{\alpha_0 + k}{\alpha_0 + \beta_0 + n} = \frac{\alpha}{\alpha + \beta}$$ where $k$ is the number of "successes" and $n$ is the number of observations where $x_i = 1$ and $\alpha_0$ and $\beta_0$ are the parameters of the prior ditribution. Given a large amount of data the choice for $\alpha_0$ and $\beta_0$ are not of much significance as we expect the results to converge to the "true" values. 




Ones we have the important fetures we want to look at wether these fetures contribute negativly or positivley.We tried to do that through computing $P(symptom=1|gene=1)$.If this probability is greater then $P(symptom=1|gene=0)=1-P(symptom=1|gene=1)$ then the effect should be negative. 

We calculated $P(symptom|gene=1)$ the bayesian way,like in 1.a. We start with i prior uniform distrbution $\alpha=1$ $\beta=1$. Update $p=\frac{\alpha}{\alpha+\beta}$.If $symptom=1$ we update $\beta$ if $symptom=0$ we update $\alpha$.

From 1.b we know that the posterior befelif of $P(symptom|gene=1)$ is given by $Beta(\alpha_{prior}+k,\beta_{prior}+n-k)$.

$k=0 , n=0$ becomes the prior dist.We update k, when we want to update $\alpha$ and $\beta$. The gene is constantly set to one,which means we are only looking at people have that gene.


These probabilities where calculated for all the relevant genes.P(symptom=1|gene=1) was around 1.5% for these genes,which could have ment that these genes have a positive effect.When we tested features that where not selected we got arouund the same procentage.The conclution is that one gene in itself have no effect on someone getting a symptom or not.

There might be such that a combination of genes together could have a postive or negative effect.This is much harder to find out,since we assume that in the data set there is not a lot of people with the same sequence of genes.

The select feature also tests $ P(symptoms|genes) \neq P(symptoms)$,separatly for each gene.

The generate binary_data function gets the inputs :N number columns,num_feutures of rows and a correlation vector.These numbers are put into the numpys random.choice functon which creats a matrix wich has dimensions N x num_feutures.Each element is eather 0 or one.This matrix is then used to creat a correlation between each column in the matrix and the target vector. The target vector is set to be a 0 vector. Each element in the target vector is flipped based  on what the value the product between a single feature vector and the random.choice is, which is 1 with p=corr and 0 with p=1-cor. Based on which feature that changes the value of the target to 1, in position i,there is now a correlation between that position in the target vector and feuture i.
Or = is used because $(1 \cup 1)=1, \ \ (1 \cup 0)=1,(0 \cup 1)=1,(0 \cup 0)=0$.  The downside with this procedure is that if one creat correlation between to many features most of the elements in the target becomes 1.


We Want to test wether the methods we use in problem 2 works as expected.To that we generate data which reflects the assumption,that if you are treated with a treatment and you had symptoms before the treatment,then after the treatment you have none. 
We therefore use the $\oplus$ operator: $$1  \  \ \ \oplus \  \  1 \ \ =0$$   
$$ = symptom before=1  \  \  \oplus  \  \ treated = symptoms \; after=1$$

 $$0 \  \  \  \oplus \  \  \ 1= \  \  1$$,   $$= (symptom before=0  \  \  \ \oplus \  \  \ treated =1) = (symptoms after=1)$$
 
 


## Estimating the efficacy of vaccines

We estimate the efficacy of the vaccines by
$$ 100 \cdot (1-IRR)$$
where IRR is the ratio of the vaccinated incidence rate and not vaccinated incidence incidenes ([link](https://www.nejm.org/doi/full/10.1056/NEJMoa2034577)). This means that the relationship we model for each symptom is as given below:

$r_v = P(symptom = 1 | vaccine = 1)$ and

$r_n = P(symptom = 1 | vaccine = 0)$

to obtain $IRR = 100 * [1 - r_v / r_n]$

Our prior belief is that r_v and r_n is equal,for all symptoms.Before we have seen the data we assume that wether you have any sumptom or not is independent from wether you have taken the vaccine or not. Theirfore we assume that these probabilities are beta distributed with the same $\alpha$ and $\beta$ parameter.To estimate these parameters we firstfind the frequency of people having a certain symptom for both the vaccinated group and the non vaccinated group and devide these frequency by the total number of observation.This is know our estimate of the expected value for r_v and r_n.So we have.

$$V=frequency \ \ \ \ of \  \ people \ \ having \ \ a \ \ certain \ \ symptom \ \ for \ \ the \ \ vaccinated$$
$$NV=frequency \ \ of \ \ people \ \  having \ \  a  \ \ certain \ \  symptom \ \ for \ \  none \ \  vaccinated$$
$$T=Total \ \ number \ \ of \ \ observations$$

$$E[r_v]=\frac{\alpha}{\alpha+\beta}=\frac{V}{T} $$ 
$$E[r_n]=\frac{\alpha}{\alpha+\beta}=\frac{NV}{T}$$

Our prior belief is that $p=\frac{V}{T}=\frac{NV}{T}$.

So:

$$E[r_v]=E[r_n]=\frac{\alpha}{\alpha+\beta}=\frac{V}{T} $$ 


The equation to solve in order to get the  $\alpha$ and the $\beta$ values for our prior belief is for example:

$$\frac{\alpha}{\alpha+\beta}=\frac{V}{T} $$

Setting $\beta=1$ we get $\alpha=\frac{\beta*p}{1-p}$.

we set $ \frac{V}{T}= 0.02 $ as our guess before seening the data. probability 0.02 for getting a symptom, indepedent of what symptom. We set beta to be 1 and $ \alpha<1$ beacause this distribution will have most of its mass around 0. In other words our prior belief is that the random variables r_v and r_n takes a value around 0 with a large probability. The probability of getting a symptom independent of vaccinated or not is very  likely to be around 0. When we get the data we can easly be proven wrong. The choosen posterior distrbution will also confurm that 0.02 is a good choice.

After we get the data 0.02 could be a horrible choice(have a low probability in the prosterior dist),and $\frac{V}{T}\neq \frac{NV}{T}$.

We have that $Posterior \propto Likelihood * Prior$.We are asuming that the prior comes from a beta distribution. The likelihhood is the likelihood of observing all the combinations of the observations.Our observations are sequences of 1 and 0's.Our likelihood is theirfore binomial dist.This makes the posterior beta dist.The prior and the posterior has the same dist.



$Posterior \propto Likelihood * Prior={n \choose k}\theta^k(1-\theta)^{n-k} \theta^{\alpha-1}(1-\theta)^{\beta-1} \propto \theta^{\alpha+k-1}(1-\theta)^{\beta+n-k-1}$ which is beta dist.




We use these posterior parameters to genrate N numbers of random variables,both for the vaccinated group and the none vaccinated group(monte carlo),in order to find a 95% credible intterval for our estimated efficacy of the vaccine on all the symptoms.

## Estimating the probability of vaccination side-effects.

Using the Bayes theorem, we will estimate the probability of vaccination side-effects. In other words, the probability of getting symptoms given that a person is vaccinated and has tested negative.
$$p_1 = P(symptom | vaccine,covid') = \frac{P(symptom \cap vaccine,covid')}{P(vaccine,covid')} $$
$$p_2 = P(symptom | vaccine',covid') = \frac{P(symptom \cap vaccine',covid')}{P(vaccine',covid')} $$

We want to test the following hypothesis:
$$ h_0: p_1 - p_2 \leq 0 $$
$$h_a: p_1 - p_2 > 0 $$

We make a confidence intervall for each symptom, and reject the null hypothesis if confidence intervall does not include zero.

**A large-sample 95% confidence interval for $p_1 - p_2$**:

In this task it seems appropiate to estimate confidence intervalls instead of doing the task of comparison through hypotesis testing.





The variables X and Y  represents the number of individuals in each sample having a certain  characteristic that defines $p_1$ and $p2$.Provided the population sizes are much larger than the sample sizes, the distribution
of X can be choosen to be binomial with parameters m and $p_1$, and similarly, Y is choosen
to be a binomial variable with parameters n and $p_2$. The samples are assumed to be independent of each other.Therefore X and Y are independent rv’s.

The estimator for $ p_1-p_2 $, the difference in population proportions, is
the  difference in sample proportions $\frac{X}{m} - \frac{Y}{n}$ .  $ \hat{p_1} = \frac{X}{m}$ and
$\hat{p_2} = \frac{Y}{m}$, the estimator of $ p_1-p_2 $ can be expressed as $\hat{p_1} - \hat{p_2}$.

$E[\hat{p_1} - \hat{p_2}]=p_1-p_2$ so $\hat{p_1} - \hat{p_2}$ is an unbiased estimate of $p_1-p_2$ 

$\hat{p_1}=\frac{X}{m}$ and $\hat{p_2}=\frac{y}{n}$ are aproximately normal distributed when m and n are large.


Refrence: Modern Mathematical Statistic with applications.

## The effect of treatments on alleviating symptoms

We are not interested in the outcomes for individuals who did not have any symptoms neither before nor after the treatment,since these individuals dont tell us anything about wether one had a positve of negative effect of the treatment.

We do this modelling task in a similar way that we did the modelling for vaccine efficacy. Since we now have data from before and after a treatment, we calculate the ratio of people who still have symptoms after treatment to the people who had symptoms before treatment for both the treated group and the control group. 



We estimate the efficacy of the treatments based on this ration


$t_1 = P(symptom = 1 | treatment = 1)$ after treatment, $t_0 = P(symptom = 1 | treatment = 1) $ before treatment.

$n_1 = P(symptom = 1 | treatment = 0)$ after treatment, $n_0 = P(symptom = 1 | treatment = 0) $ before treatment

$r_t = t_1 / t_0$ and

$r_n = n_1 / n_0$


$$ ratio=\frac{r_n}{r_t} $$

## The modelling

We start by importing the libraries and loading the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.special import btdtri
import random
#from sklearnex import patch_sklearn
#patch_sklearn()
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline
import statsmodels.api as sm

In [2]:
obs_data = pd.read_csv("observation_features.csv")
treat_data = pd.read_csv("treatment_features.csv")
action_data = pd.read_csv("treatment_actions.csv")
outcome_data = pd.read_csv("treatment_outcomes.csv")

cols = (['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death',
        'Age', 'Gender', 'Income'] +
         [f'Gene_{i+1:03}' for i in range(128)] +
         ['Asthma', 'Obesity', 'Smoking', 'Diabetes', 'Heart disease', 'Hypertension',
         'Vacc_1', 'Vacc_2', 'Vacc_3'])

obs_data.columns = cols
treat_data.columns = cols
outcome_data.columns = cols[:10]
action_data.columns = ['Treatment_1', 'Treatment_2']

## The pipelines

In this section we indroduce **Pipeline_observational**. This pipeline will be used throughout problem. It contains the methods for modelling the effective genes, finding efficacy of the vaccines and the estimating the probability of sideeffects for all vaccines. We also introduce the **Pipeline_treatment** which contains methods for estimating the efficacy for treatments.

The pipelines will be run with generated data and tested before we 

In [4]:
class Pipeline_observational():

    def __init__(self, obs_data=None):
        self.obs_data = obs_data
        self.symptoms = obs_data.iloc[:,:10]
        self.genes = obs_data.iloc[:, 10:139]
        #self.vaccine = obs_data[:,147:]

    def run_select_features(self, genes, symptom, threshold):
        """This function finds the selected features, then runs BIC test in order 
        to see whether the model with selected features are better than the full model"""
        
        self.best_features = best_features = self.select_features(genes,symptom,threshold)
        self.important_genes = [col for col in genes.iloc[:,best_features].columns]

    def select_features(self, genes, symptoms, threshold):
        """ Select the most important features of a data set, where X (2D)
        contains the feature data, and Y (1D) contains the target
        """
        X, Y = np.array(genes), np.array(symptoms)
        n_features = X.shape[1]
        n_data =  X.shape[0]
        alpha_b = np.ones([n_features, 2 ])
        beta_b = np.ones([n_features, 2])
        log_p = np.zeros(n_features)

        log_null = 0
        alpha = 1
        beta = 1
        for t in range(n_data):
            p_null = alpha / (alpha + beta)
            log_null += np.log(p_null)*Y[t] + np.log(1-p_null)*(1 - Y[t])
            alpha += Y[t]
            beta += (1 - Y[t])
            for i in range(n_features):
                x_ti = int(X[t,i])
                p = alpha_b[i, x_ti] / (alpha_b[i, x_ti] + beta_b[i, x_ti])
                log_p[i] += np.log(p)*Y[t] + np.log(1-p)*(1 - Y[t])
                alpha_b[i, x_ti] += Y[t]
                beta_b[i, x_ti] += (1 - Y[t])
        log_max=np.mean(log_p)
        log_max2=np.mean(log_null)
        log_p=log_p-log_max
        log_null=log_null-log_max2
        p = 1 / (np.exp(log_null - log_p) + 1)
        return [i for i in range(n_features) if p[i] > threshold]
    
    def estimate_gene_effect(self,symptom, genome):
        """Given a genome and a symptom, we estimate the probability of having a symptom from the data"""
        symptom, genome = np.array(symptom), np.array(genome)
        alpha = 1
        beta = 1
        for t in range(genome.shape[0]):
            if genome[t] == 1:
                alpha += symptom[t]
                beta += (1 - symptom[t])

        return alpha/(alpha+beta)

    def estimate_vaccine_efficacy(self, group_pos: pd.DataFrame, group_neg: pd.DataFrame, symptom, prior_probs):
        """ Given a dataframe group_pos (vaccinated and covid positive) and group_neg (not vaccinated and covid postive) we estimate the efficacy of the vaccines"""
        
        if isinstance(symptom, int):
            symptom_index = symptom
            symptom_name = group_pos.keys()[symptom]
        else:
            symptom_name = symptom
            symptom_index = list(group_pos.keys()).index(symptom)

        group_pos_count = np.sum(group_pos[symptom_name] * group_pos.iloc[:,1])
        group_neg_count = np.sum(group_neg[symptom_name] * group_neg.iloc[:,1])

        v = group_pos_count/len(group_pos)
        n_v = group_neg_count/len(group_neg)

        if n_v == 0:
            print(f'{v}, {n_v}: Division by zero')
            return

        IRR = v/n_v

        efficacy = 100*(1- IRR)

        N = 100_000
        beta = 1
        p = prior_probs[symptom_index]
        alpha = beta*p/(1-p)

        samples_group_pos = stats.beta.rvs(alpha + group_pos_count, beta + len(group_pos) - group_pos_count, size=N)
        samples_group_neg = stats.beta.rvs(alpha + group_neg_count, beta + len(group_neg) - group_neg_count, size=N)

        samples_ve = 100 * (1 - samples_group_pos/samples_group_neg)
        lower = np.percentile(samples_ve, 2.5)
        upper = np.percentile(samples_ve, 97.5)
    
        print(f'{symptom_name:15s}: {efficacy:3.3f} - ({lower:3.3f}, {upper:3.3f})')
    
    def print_vaccine_efficacy(self, symptoms: pd.DataFrame, vaccinated: pd.DataFrame, not_vaccinated: pd.DataFrame):
        """Given a set of vaccinated and unvaccinated data, we estimate the efficacy of all vaccines together and then one by one """
        for i, s in enumerate(symptoms.columns):
            self.estimate_vaccine_efficacy(vacced,un_vacced,i,prior_probs)
        print("")
        

        vaccination_types = [vaccinated[vaccinated.iloc[:,v] == 1].iloc[:,v] for v in range(vaccinated.shape[1])]
        vaccination_types.colums = [f'vaccine{i}' for i in range(vaccinated.shape[1])]
        
        for i, vaccine in enumerate(vaccination_types):
            print(vaccination_types.columns[i])
            for j, s in enumerate(symptoms.columns):
                self.find_vaccine_efficacy(vaccination_types[j],not_vaccinated,s,prior_probs)
            print("")
        
        
    def estimate_side_effects(self, vacced_neg, un_vacced_neg, start, end):
        """Given a set of vaccinated and unvaccinated data, we estimate the efficacy of the vaccines"""

        stats = pd.DataFrame(index=vacced_neg.keys()[start:end],
                          columns = ("p1 (%)", "p2 (%)", "Diff (%)", "Confidence Interval (%)", "Null Hypothesis", ),
                         )

        for i in range(start, end):
            symptom = vacced_neg.keys()[i]
            p1 = vacced_neg.sum()[i] / vacced_neg.shape[0]
            p2 = un_vacced_neg.sum()[i] / un_vacced_neg.shape[0]


            lower = (p1-p2 - 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))
            higher = (p1-p2 + 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))

            p1, p2, lower, higher = p1 * 100, p2 * 100, lower * 100, higher * 100

            stats.loc[symptom] = np.array([round(p1, 4), round(p2, 4), round(p1 - p2, 4), (round(lower, 4), round(higher, 4)),
                               "rejected" if lower>0 else "not rejected", ],dtype=object)


        return stats

### Treat

In [9]:
class Pipeline_treatment():
    
    def __init__(self,treat_data,action_data,outcome_data):
        """Given the pre treatment symptoms, the treatment action and post treatment symptoms, we slice the data into actionstypes and outcome types """

        import warnings
        warnings.filterwarnings('ignore')

        self.treat_data = treat_data
        self.action_data = action_data
        self.outcome_data = outcome_data

        new_treat_data = treat_data[((np.sum(treat_data.iloc[:,2:10],axis=1) > 0.0) | np.sum(outcome_data.iloc[:,2:10],axis=1) > 0.0)]
        self.group_first = new_treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
        self.group_second = new_treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
        self.group_both = new_treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
        self.group_none = new_treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]

        new_outcome_data = outcome_data[((np.sum(treat_data.iloc[:,2:10],axis=1) > 0.0) | np.sum(outcome_data.iloc[:,2:10],axis=1) > 0.0)]
        self.outcome_first = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
        self.outcome_second = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
        self.outcome_both = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
        self.outcome_none = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]
       
        self.prior_probs= [(np.sum(new_treat_data[sym]) + np.sum(new_outcome_data[sym])) / (len(new_treat_data) * 2) for sym in outcome_data.columns][2:]

    def print_treatment_efficacy(self):
        outcome_first, outcome_second, outcome_both, outcome_none, group_first, group_second, group_both, group_none = self.outcome_first, self.outcome_second, self.outcome_both, self.outcome_none, self.group_first, self.group_second, self.group_both, self.group_none
        print(self.prior_probs)
        
        for outcome_treated, pre_treated, treatment in zip([outcome_first, outcome_second, outcome_both],[group_first, group_second, group_both],['Treatment 1', 'Treatment 2', 'Both treatments']):
            print(f"{treatment} efficacy:")
            results = {}
            for i, key in enumerate(self.outcome_data.keys()[2:]):
                eff, ci = self.treatment_efficacy(outcome_treated, pre_treated, outcome_none, group_none, self.prior_probs[i], key,log=False)
                results[key] = eff; results['95% CI'] = ci
            
            print(pd.DataFrame(results).T)


    def find_alpha(self, beta,p):
        """ Given beta and a mean probability p, compute and return the alpha of a beta distribution. """
        return beta*p/(1-p)

    def treatment_efficacy(self, outcome_treated, precondition_treated, outcome_untreated, precondition_untreated, p, symptom_name, log=True):
        
        group_pos_count = np.sum(outcome_treated[symptom_name])
        group_neg_count = np.sum(outcome_untreated[symptom_name])

        group_pos_total = np.sum(precondition_treated[symptom_name])
        group_neg_total = np.sum(precondition_untreated[symptom_name])

        if any(v == 0 for v in (group_pos_total, group_neg_total, group_neg_count)):
            print(f'{symptom_name:15s}: Division by zero - not enough data to compute efficacy' )
            return

        v = group_pos_count / group_pos_total
        n_v = group_neg_count / group_neg_total
        IRR = v/n_v

        efficacy = 100 * (1- IRR)

        N=100_000
        beta = 1
        alpha = beta*p/(1-p)

        samples_group_pos = stats.beta.rvs(alpha + group_pos_count, beta + len(outcome_treated) - group_pos_count, size=N)
        samples_group_neg = stats.beta.rvs(alpha + group_neg_count, beta + len(outcome_untreated) - group_neg_count, size=N)

        samples_ve = 100 * (1 - samples_group_pos/samples_group_neg)
        lower = np.percentile(samples_ve, 2.5)
        upper = np.percentile(samples_ve, 97.5)
        

        
        if log is True:
            print(f'{symptom_name:15s}: {efficacy:7.3f} - 95% CI: ({lower:3.3f}, {upper:3.3f})')

        return efficacy, (lower, upper)
    
    
    def plot_symptoms(self):
        pre_treatment = {}
        treatment_one = {}
        treatment_two = {}
        treatment_both = {}
        treatment_none = {}

        for symptom_name in self.treat_data.columns[2:10]:
            pre_treatment[symptom_name] = np.sum(self.treat_data[symptom_name])
            treatment_one[symptom_name] = np.sum(self.outcome_first[symptom_name])
            treatment_two[symptom_name] = np.sum(self.outcome_second[symptom_name])
            treatment_both[symptom_name] = np.sum(self.outcome_both[symptom_name])
            treatment_none[symptom_name] = np.sum(self.outcome_none[symptom_name])

        x_axis = np.arange(len(pre_treatment.keys()))
        plt.bar(x_axis-0.24, pre_treatment.values(), width = 0.12, color='b', label = "Pre treatment")
        plt.bar(x_axis-0.12, treatment_one.values(), width = 0.12, color='g', label = "Treatment 1")
        plt.bar(x_axis, treatment_two.values(), width = 0.12,color='r', label = "Treatment 2")
        plt.bar(x_axis+0.12, treatment_both.values(), width = 0.12,color='purple', label = "Both treatments")
        plt.bar(x_axis+0.24, treatment_none.values(), width = 0.12,color='pink', label = "No treatment")
        plt.xticks(x_axis, self.treat_data.columns[2:10], rotation=90)
        plt.ylabel("Number of occurances")
        plt.legend()
        plt.show()

Most of the the individuals did'nt have any symptoms. We can theirfore see that the classifier we used failed to classify all the individuals that did have symptoms.The only way one can then determain if our select feuture method works or not is using generated data,that are highly squed toward individuals having symptoms.

Our result also shows that genes in them selves are not very good determiners for wether somebody that are covid positive gets a certain symptom or not. These types of conclutions can only be made using the original data. 

To see if their is somthing "wrong" with the algorithm one could try different algorithms.If the result still is the same, 
genes in them selves are not very good determiners for wether somebody that are covid positive gets a certain symptom or not.


Only use generated data to confirm wether the select feuture method works as it is expected to work.You cant alter reality as you want,generating data to force feutures to be significant,does not reflect reality,it reflects what you want reality to be.

Could test gender and comorbidities which also is binary variables



## Generating synthetic data

In [22]:
def generate_binary_data(num_features, N, correlation=[0.9, 0.5]):
    data = np.random.choice(2, size=(N, num_features))
    df = pd.DataFrame(data)
    df["Target"] = np.zeros(N).astype(int)
    for i, cor in enumerate(correlation):
        if i >= num_features:
            break

        df["Target"] |= df.iloc[:, i] * np.random.choice(2, size=N, p=[(1-cor), cor])

    return df.iloc[:, :num_features], df["Target"]

def generate_genomes_symptoms(random_indecies):
    cor = [0.2 for _ in range(128)]
    for r in random_indecies:
        cor[r] = 0.9
    X,y = generate_binary_data(128,100_000, correlation=cor)
    X.columns = [f'Gene_{i+1:03}' for i in range(128)]
    y.columns = cols[1]
    return X,y   

In [20]:
def generate_vaccine_data():
    sym_g = np.random.choice(2,size=[100_000,10])
    y = np.random.choice(2,size=[100_000,1])

    vac_g = np.zeros([100_000,3])
    for i,v in enumerate(y):
        if v == 1:
            rand_ind = random.randint(0,2)
            vac_g[i][rand_ind] = 1

    vac_g = pd.DataFrame(vac_g,columns = ['Vacc_1', 'Vacc_2', 'Vacc_3'])
    sym_g = pd.DataFrame(sym_g,columns = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death'])

    vacced = sym_g[np.sum(vac_g.iloc[:,-3:], axis=1) == 1]
    un_vacced = sym_g[np.sum(vac_g.iloc[:,-3:], axis=1) == 0]
    prior_probs_generated = [np.sum(sym_g.iloc[:,i]) / len(sym_g) for i, key in enumerate(sym_g.columns)]
    return vacced, un_vacced, prior_probs_generated

### The experiment setup

In this section we will set up the experiment by running the pipeline with different generated data and see whether it works.

In [28]:
for i in range(10):
    print(f'Run nr: {i}')
    random_indecies = random.sample(range(128), 20)
    genomes,symptom = generate_genomes_symptoms(random_indecies)
    pipe = Pipeline_observational()
    pipe.run_select_features(genomes,symptom,0.8)
    
    for i in pipe.best_features:
        if i not in random_indecies:
            print(f'{i}: fail')


    #kjøre efficacy
    #kjøre sideeffects

Run nr: 0
Run nr: 1


KeyboardInterrupt: 

## Running pipeline on real data

In [10]:
pipe_t = Pipeline_treatment(treat_data, action_data, outcome_data)

pipe_t.print_treatment_efficacy()
pipe_t.plot_symptoms()

[0.21951219512195122, 0.12804878048780488, 0.024390243902439025, 0.16158536585365854, 0.024390243902439025, 0.057926829268292686, 0.15548780487804878, 0.054878048780487805]
Treatment 1 efficacy:
Stomach        : Division by zero - not enough data to compute efficacy


TypeError: 'NoneType' object is not iterable

## Conclusion

In [25]:
pipe_o = Pipeline_observational(obs_data)

vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 1]
vacced_neg = vacced[vacced.iloc[:,1]==0]
vacced_pos = vacced[vacced.iloc[:,1]==1]

un_vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 0]
un_vacced_neg = un_vacced[un_vacced.iloc[:,1]==0]
un_vacced_pos = un_vacced[un_vacced.iloc[:,1]==1]
prior_probs= [np.sum(obs_data.iloc[:,i]) / len(obs_data) for i, key in enumerate(obs_data.iloc[:,2:10].columns)]


MemoryError: Unable to allocate 68.6 MiB for an array with shape (150, 59963) and data type float64

In [64]:
symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death']
symptoms = obs_data.iloc[:,2:10]
genes = obs_data.iloc[:, 10:139]

for s in symptom_names[2:10]:
    print(s)
    relevant_genes = pipe_o.select_features(genes, symptoms, 0.8)
    print(f'Relevant genes: {relevant_genes}')
    print([f'Probability of getting {s} if {relevant_genes[i]}: ' + {pipe_o.estimate_gene_effect(symptoms[s], genes.iloc[relevant_genes[i]])} for i in enumerate(relevant_genes)])

pipe_o.run_vaccine_efficacy(obs_data.iloc[:,2:10], vacced, un_vacced)
pipe_o.estimate_side_effects(vacced_neg, un_vacced_neg, 1, 10)



No-Taste/Smell


IndexError: index 9 is out of bounds for axis 1 with size 2